In [ ]:
%run './remodelisations/communes.ipynb'
%run './remodelisations/dep.ipynb'
%run './remodelisations/regions.ipynb'
%run './remodelisations/statistiques.ipynb' # les stats ont des problèmes avec les fk -> ref vers des code de communes inconnues

In [1]:
import psycopg2 as postgres

In [2]:
BASE = 'tp_dba'
USER = 'axelmoriceau'
PSWD = ''

conn = postgres.connect(
    host='localhost',
    database=BASE,
    user=USER,
    password=PSWD,
    port='5432'
)

cur = conn.cursor()


In [3]:
tables = ['departements_cheflieu','regions_cheflieu', 'communes', 'departements', 'regions','indicateurs','statistiques']

In [4]:
# A enlever avant rendu 

drop_tables = ''
for table in tables:
    drop_tables += f'DROP TABLE IF EXISTS {table} CASCADE;'


In [5]:
cur.execute(drop_tables)
conn.commit()

In [6]:
cur.execute(open("sql/create.sql", "r").read())
conn.commit()

In [7]:
files = [
        'regions', 
        'departements',
        'communes',
        'departements_cheflieu',
        'regions_cheflieu',
        'indicateurs',
        'statistiques'
    ]

for file in files:
    path = f'./csv/filtered/{file}.csv'
    f = open(path, 'r')
    cur.copy_from(f, file, sep=',', null='None')
    f.close()
    conn.commit()


In [8]:
conn.close()